In [8]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path

In [9]:
### Read all the pdf's inside the directory
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data")
all_pdf_documents

Found 4 PDF files to process

Processing: 10700122071_Puspita Panja (2).pdf
  ✓ Loaded 8 pages

Processing: 10700122071_Puspita Panja.pdf
  ✓ Loaded 7 pages

Processing: 10700122071_Puspita_Panja.pdf
  ✓ Loaded 7 pages

Processing: remotesensing-18-00203.pdf
  ✓ Loaded 29 pages

Total documents loaded: 51


[Document(metadata={'producer': 'Microsoft: Print To PDF', 'creator': 'PyPDF', 'creationdate': '2025-08-25T17:05:13+05:30', 'author': '', 'moddate': '2025-08-25T17:05:13+05:30', 'title': 'Microsoft Word - Document1', 'source': '..\\data\\text_files\\10700122071_Puspita Panja (2).pdf', 'total_pages': 8, 'page': 0, 'page_label': '1', 'source_file': '10700122071_Puspita Panja (2).pdf', 'file_type': 'pdf'}, page_content='COLLEGE OF ENGINEERING AND \nMANAGEMENT, KOLAGHAT \n \n \n \nNAME: - PUSPITA PANJA \nCOLLEGE ROLL NUMBER: - CSE/22/065 \nSUBJECT: - MACHINE LEARNING  \nSUBJECT CODE: - PEC-CS701E \nUNIVERSITY RAGISTRAION NO.: - \n221070110076 \nUNIVERSITY ROLL NO.: - 10700122071 \nDEPARTMENT: - COMPUTER SCIENCE AND \nENGINEERING \nSECTION: - ‘C’ \nACADEMIC SESSION: - 2022-2026'),
 Document(metadata={'producer': 'Microsoft: Print To PDF', 'creator': 'PyPDF', 'creationdate': '2025-08-25T17:05:13+05:30', 'author': '', 'moddate': '2025-08-25T17:05:13+05:30', 'title': 'Microsoft Word - Document

In [10]:
### Text splitting get into chunks

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs

In [11]:
chunks=split_documents(all_pdf_documents)
chunks

Split 51 documents into 167 chunks

Example chunk:
Content: COLLEGE OF ENGINEERING AND 
MANAGEMENT, KOLAGHAT 
 
 
 
NAME: - PUSPITA PANJA 
COLLEGE ROLL NUMBER: - CSE/22/065 
SUBJECT: - MACHINE LEARNING  
SUBJECT CODE: - PEC-CS701E 
UNIVERSITY RAGISTRAION NO.: ...
Metadata: {'producer': 'Microsoft: Print To PDF', 'creator': 'PyPDF', 'creationdate': '2025-08-25T17:05:13+05:30', 'author': '', 'moddate': '2025-08-25T17:05:13+05:30', 'title': 'Microsoft Word - Document1', 'source': '..\\data\\text_files\\10700122071_Puspita Panja (2).pdf', 'total_pages': 8, 'page': 0, 'page_label': '1', 'source_file': '10700122071_Puspita Panja (2).pdf', 'file_type': 'pdf'}


[Document(metadata={'producer': 'Microsoft: Print To PDF', 'creator': 'PyPDF', 'creationdate': '2025-08-25T17:05:13+05:30', 'author': '', 'moddate': '2025-08-25T17:05:13+05:30', 'title': 'Microsoft Word - Document1', 'source': '..\\data\\text_files\\10700122071_Puspita Panja (2).pdf', 'total_pages': 8, 'page': 0, 'page_label': '1', 'source_file': '10700122071_Puspita Panja (2).pdf', 'file_type': 'pdf'}, page_content='COLLEGE OF ENGINEERING AND \nMANAGEMENT, KOLAGHAT \n \n \n \nNAME: - PUSPITA PANJA \nCOLLEGE ROLL NUMBER: - CSE/22/065 \nSUBJECT: - MACHINE LEARNING  \nSUBJECT CODE: - PEC-CS701E \nUNIVERSITY RAGISTRAION NO.: - \n221070110076 \nUNIVERSITY ROLL NO.: - 10700122071 \nDEPARTMENT: - COMPUTER SCIENCE AND \nENGINEERING \nSECTION: - ‘C’ \nACADEMIC SESSION: - 2022-2026'),
 Document(metadata={'producer': 'Microsoft: Print To PDF', 'creator': 'PyPDF', 'creationdate': '2025-08-25T17:05:13+05:30', 'author': '', 'moddate': '2025-08-25T17:05:13+05:30', 'title': 'Microsoft Word - Document

In [12]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager
        
        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


## initialize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2
Model loaded successfully. Embedding dimension: 384


In [14]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore
    

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 295


In [15]:
chunks

[Document(metadata={'producer': 'Microsoft: Print To PDF', 'creator': 'PyPDF', 'creationdate': '2025-08-25T17:05:13+05:30', 'author': '', 'moddate': '2025-08-25T17:05:13+05:30', 'title': 'Microsoft Word - Document1', 'source': '..\\data\\text_files\\10700122071_Puspita Panja (2).pdf', 'total_pages': 8, 'page': 0, 'page_label': '1', 'source_file': '10700122071_Puspita Panja (2).pdf', 'file_type': 'pdf'}, page_content='COLLEGE OF ENGINEERING AND \nMANAGEMENT, KOLAGHAT \n \n \n \nNAME: - PUSPITA PANJA \nCOLLEGE ROLL NUMBER: - CSE/22/065 \nSUBJECT: - MACHINE LEARNING  \nSUBJECT CODE: - PEC-CS701E \nUNIVERSITY RAGISTRAION NO.: - \n221070110076 \nUNIVERSITY ROLL NO.: - 10700122071 \nDEPARTMENT: - COMPUTER SCIENCE AND \nENGINEERING \nSECTION: - ‘C’ \nACADEMIC SESSION: - 2022-2026'),
 Document(metadata={'producer': 'Microsoft: Print To PDF', 'creator': 'PyPDF', 'creationdate': '2025-08-25T17:05:13+05:30', 'author': '', 'moddate': '2025-08-25T17:05:13+05:30', 'title': 'Microsoft Word - Document

In [16]:
### Convert the text to embeddings
texts=[doc.page_content for doc in chunks]

## Generate the Embeddings

embeddings=embedding_manager.generate_embeddings(texts)

##store in the vector database
vectorstore.add_documents(chunks,embeddings)

Generating embeddings for 167 texts...


Batches: 100%|██████████| 6/6 [00:06<00:00,  1.02s/it]


Generated embeddings with shape: (167, 384)
Adding 167 documents to vector store...
Successfully added 167 documents to vector store
Total documents in collection: 462


In [18]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""
    
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever
        
        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
            
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")
        
        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        
        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            
            # Process results
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            
            return retrieved_docs
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever=RAGRetriever(vectorstore,embedding_manager)
rag_retriever

In [19]:
rag_retriever.retrieve("Working Principle of Dimensionality Reduction")

Retrieving documents for query: 'Working Principle of Dimensionality Reduction'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00,  8.91it/s]

Generated embeddings with shape: (1, 384)
Retrieved 5 documents (after filtering)


[{'id': 'doc_9e6b34ad_1',
  'content': 'Table of Contents: \n\uf0b7 Introduction to Dimensionality Reduction \n\uf0b7 Working Principle of Dimensionality Reduction \n\uf0b7 Dimensionality Reduction Techniques \n\uf0b7 Dimensionality Reduction Real World Examples \n\uf0b7 Advantages of Dimensionality Reduction \n\uf0b7 Disadvantages of Dimensionality Reduction \n\uf0b7 Conclusion \n\uf0b7 References',
  'metadata': {'producer': 'Microsoft: Print To PDF',
   'total_pages': 8,
   'author': '',
   'creationdate': '2025-08-25T17:05:13+05:30',
   'page_label': '2',
   'source': '..\\data\\text_files\\10700122071_Puspita Panja (2).pdf',
   'content_length': 313,
   'source_file': '10700122071_Puspita Panja (2).pdf',
   'creator': 'PyPDF',
   'moddate': '2025-08-25T17:05:13+05:30',
   'file_type': 'pdf',
   'page': 1,
   'title': 'Microsoft Word - Document1',
   'doc_index': 1},
  'similarity_score': 0.5127190351486206,
  'distance': 0.4872809648513794,
  'rank': 1},
 {'id': 'doc_6ee1be19_1',


In [20]:
rag_retriever.retrieve("What is trapdoor?")

Retrieving documents for query: 'What is trapdoor?'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 11.94it/s]

Generated embeddings with shape: (1, 384)
Retrieved 5 documents (after filtering)


[{'id': 'doc_a1a0cb9a_24',
  'content': 'Table of Contents:•Introduction to Trapdoor•Working Procedure•Different Types of Trapdoors•Functions of trap doors in cyber security•Conclusion•References',
  'metadata': {'source': '..\\data\\text_files\\10700122071_Puspita_Panja.pdf',
   'creator': 'PyPDF',
   'file_type': 'pdf',
   'content_length': 154,
   'producer': 'Microsoft: Print To PDF',
   'title': 'Microsoft PowerPoint - CS.pptx',
   'page': 1,
   'page_label': '2',
   'source_file': '10700122071_Puspita_Panja.pdf',
   'moddate': '2025-08-02T13:33:11+05:30',
   'total_pages': 7,
   'creationdate': '2025-08-02T13:33:11+05:30',
   'author': '',
   'doc_index': 24},
  'similarity_score': 0.37390339374542236,
  'distance': 0.6260966062545776,
  'rank': 1},
 {'id': 'doc_b7b01a14_24',
  'content': 'Table of Contents:•Introduction to Trapdoor•Working Procedure•Different Types of Trapdoors•Functions of trap doors in cyber security•Conclusion•References',
  'metadata': {'total_pages': 7,
   

In [21]:
### Simple RAG pipeline with Groq LLM
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()

### Initialize the Groq LLM (set your GROQ_API_KEY in environment)
groq_api_key = os.getenv("GROQ_API_KEY")
llm=ChatGroq(groq_api_key=groq_api_key,model_name="llama-3.1-8b-instant",temperature=0.1,max_tokens=1024)

## 2. Simple RAG function: retrieve context + generate response
def rag_simple(query,retriever,llm,top_k=3):
    ## retriever the context
    results=retriever.retrieve(query,top_k=top_k)
    context="\n\n".join([doc['content'] for doc in results]) if results else ""
    if not context:
        return "No relevant context found to answer the question."
    
    ## generate the answwer using GROQ LLM
    prompt=f"""Use the following context to answer the question concisely.
        Context:
        {context}

        Question: {query}

        Answer:"""
    
    response=llm.invoke([prompt.format(context=context,query=query)])
    return response.content

In [22]:
answer=rag_simple("what is Digital Replica ?",rag_retriever,llm)
print(answer)


Retrieving documents for query: 'what is Digital Replica ?'
Top K: 3, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 13.49it/s]

Generated embeddings with shape: (1, 384)
Retrieved 2 documents (after filtering)


Digital Replica (DR) is an approach that combines reality-based models, high visual quality, and interactivity within an Open World, providing effective tools for study and dissemination.


In [23]:
answer=rag_simple("examples of Magnetic Disk",rag_retriever,llm)
print(answer)

Retrieving documents for query: 'examples of Magnetic Disk'
Top K: 3, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 12.89it/s]

Generated embeddings with shape: (1, 384)
Retrieved 3 documents (after filtering)


Features of Magnetic Disk (page 1)


In [24]:
# --- Enhanced RAG Pipeline Features ---
def rag_advanced(query, retriever, llm, top_k=5, min_score=0.2, return_context=False):
    """
    RAG pipeline with extra features:
    - Returns answer, sources, confidence score, and optionally full context.
    """
    results = retriever.retrieve(query, top_k=top_k, score_threshold=min_score)
    if not results:
        return {'answer': 'No relevant context found.', 'sources': [], 'confidence': 0.0, 'context': ''}
    
    # Prepare context and sources
    context = "\n\n".join([doc['content'] for doc in results])
    sources = [{
        'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
        'page': doc['metadata'].get('page', 'unknown'),
        'score': doc['similarity_score'],
        'preview': doc['content'][:300] + '...'
    } for doc in results]
    confidence = max([doc['similarity_score'] for doc in results])
    
    # Generate answer
    prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"""
    response = llm.invoke([prompt.format(context=context, query=query)])
    
    output = {
        'answer': response.content,
        'sources': sources,
        'confidence': confidence
    }
    if return_context:
        output['context'] = context
    return output


# Example usage:
result = rag_advanced(" The Excavation for the Amba Aradam Station of Rome’s Metro C:AnUrbanArchaeology Project", rag_retriever, llm, top_k=3, min_score=0.1, return_context=True)
print("Answer:", result['answer'])
print("Sources:", result['sources'])
print("Confidence:", result['confidence'])
print("Context Preview:", result['context'][:300])

Retrieving documents for query: ' The Excavation for the Amba Aradam Station of Rome’s Metro C:AnUrbanArchaeology Project'
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 11.54it/s]

Generated embeddings with shape: (1, 384)
Retrieved 3 documents (after filtering)


Answer: The Excavation for the Amba Aradam Station of Rome's Metro C: An Urban Archaeology Project.
Sources: [{'source': 'remotesensing-18-00203.pdf', 'page': 3, 'score': 0.7163557410240173, 'preview': 'particularly regarding temporal integration workflows and reproducible quality standards\nfor immersive visualization.\n3. The Excavation for the Amba Aradam Station of Rome’s Metro C:\nAn Urban Archaeology Project\nThe archaeological investigations at the Amba Aradam station were carried out in\nan area...'}, {'source': 'remotesensing-18-00203.pdf', 'page': 3, 'score': 0.7163557410240173, 'preview': 'particularly regarding temporal integration workflows and reproducible quality standards\nfor immersive visualization.\n3. The Excavation for the Amba Aradam Station of Rome’s Metro C:\nAn Urban Archaeology Project\nThe archaeological investigations at the Amba Aradam station were carried out in\nan area...'}, {'source': 'remotesensing-18-00203.pdf', 'page': 1, 'score': 0.3927212953567505,

In [25]:
# --- Advanced RAG Pipeline: Streaming, Citations, History, Summarization ---
from typing import List, Dict, Any
import time

class AdvancedRAGPipeline:
    def __init__(self, retriever, llm):
        self.retriever = retriever
        self.llm = llm
        self.history = []  # Store query history

    def query(self, question: str, top_k: int = 5, min_score: float = 0.2, stream: bool = False, summarize: bool = False) -> Dict[str, Any]:
        # Retrieve relevant documents
        results = self.retriever.retrieve(question, top_k=top_k, score_threshold=min_score)
        if not results:
            answer = "No relevant context found."
            sources = []
            context = ""
        else:
            context = "\n\n".join([doc['content'] for doc in results])
            sources = [{
                'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
                'page': doc['metadata'].get('page', 'unknown'),
                'score': doc['similarity_score'],
                'preview': doc['content'][:120] + '...'
            } for doc in results]
            # Streaming answer simulation
            prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {question}\n\nAnswer:"""
            if stream:
                print("Streaming answer:")
                for i in range(0, len(prompt), 80):
                    print(prompt[i:i+80], end='', flush=True)
                    time.sleep(0.05)
                print()
            response = self.llm.invoke([prompt.format(context=context, question=question)])
            answer = response.content

        # Add citations to answer
        citations = [f"[{i+1}] {src['source']} (page {src['page']})" for i, src in enumerate(sources)]
        answer_with_citations = answer + "\n\nCitations:\n" + "\n".join(citations) if citations else answer

        # Optionally summarize answer
        summary = None
        if summarize and answer:
            summary_prompt = f"Summarize the following answer in 2 sentences:\n{answer}"
            summary_resp = self.llm.invoke([summary_prompt])
            summary = summary_resp.content

        # Store query history
        self.history.append({
            'question': question,
            'answer': answer,
            'sources': sources,
            'summary': summary
        })

        return {
            'question': question,
            'answer': answer_with_citations,
            'sources': sources,
            'summary': summary,
            'history': self.history
        }

# Example usage:
adv_rag = AdvancedRAGPipeline(rag_retriever, llm)
result = adv_rag.query("Features of Magnetic Disk", top_k=3, min_score=0.1, stream=True, summarize=True)
print("\nFinal Answer:", result['answer'])
print("Summary:", result['summary'])
print("History:", result['history'][-1])

Retrieving documents for query: 'Features of Magnetic Disk'
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00,  6.69it/s]

Generated embeddings with shape: (1, 384)
Retrieved 3 documents (after filtering)
Streaming answer:
Use the following context to answer the question concisely.
Context:
TABLE OF CO

NTENTS  
 
TOPIC                                                                                    PAGE 
Introduction     1 
Magnetic Disk    1 
Features of Magnetic Disk    1 
Optical Disk    1 
Features of Optical Disk    2 
Difference in Magnetic Disk and Optical Disk    2 
Conclusion    3 
Reference    3

TABLE OF CONTENTS  
 
TOPIC                                                                                    PAGE 
Introduction     1 
Magnetic Disk    1 
Features of Magnetic Disk    1 
Optical Disk    1 
Features of Optical Disk    2 
Difference in Magnetic Disk and Optical Disk    2 
Conclusion    3 
Reference    3

TABLE OF CONTENTS  
 
TOPIC                                                                                    PAGE 
Introduction     1 
Magnetic Disk    1 
Features of Magnetic Disk    1 
Optical Disk    1 
Features of Optical Disk    2 
Difference in Magnetic Disk and Optical Disk    2 
Conclusion    3 
Reference    3

Question: Features of Magnetic Disk

Answe